In [ ]:
!pwd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import sys
import itertools
from collections import Counter, defaultdict
import numpy as np
import scipy.stats, scipy.spatial, scipy.signal
import os
import json
from path import Path; Path.stem = Path.namebase
from natsort import natsorted

from ppm3d import Cluster, align, AlignedData
from motifextraction import load_cns
from motifextraction.utils import get_norm_factors

In [2]:
# Set up some paths to load data from
# The motifs I used here were just to do some basic testing.

temperature_path = Path("../")
T = 0
data_path = temperature_path / f"{T}" / Path("metallic_glass/data")
cluster_path = data_path / "clusters"
motif_path = Path("../motifextraction/examples/motifextraction/metallic_glass/data/averaged/")
motif_errors_path = data_path / "motif_errors"

In [4]:
cluster_cns = load_cns(cluster_path).astype(int)
NCLUSTERS = len(cluster_cns)
print(NCLUSTERS)

Loading cns...
9826


In [16]:
# Load norm factors so we can correctly normalize the motif errors.
# The norm factors are calculated using the all-to-all alignments.
with open(motif_path / "../norm_factors.json") as f:
    _norm_factors = json.load(f)
    L2_norm_factors = _norm_factors["L2"]
    L1_norm_factors = _norm_factors["L1"]
    Linf_norm_factors = _norm_factors["Linf"]
    angular_norm_factors = _norm_factors["angular"]

In [20]:
def calculate_gmean(errors):
    errors[:, 0][np.where(errors[:, 0] > L2_norm_factors['set_to_inf_before_dividing'])] = np.inf
    errors[:, 0][np.isinf(errors[:, 0])] = np.nan
    errors[:, 0] /= L2_norm_factors['divide_by']
    errors[:, 1][np.where(errors[:, 1] > L1_norm_factors['set_to_inf_before_dividing'])] = np.inf
    errors[:, 1][np.isinf(errors[:, 1])] = np.nan
    errors[:, 1] /= L1_norm_factors['divide_by']
    errors[:, 2][np.where(errors[:, 2] > Linf_norm_factors['set_to_inf_before_dividing'])] = np.inf
    errors[:, 2][np.isinf(errors[:, 2])] = np.nan
    errors[:, 2] /= Linf_norm_factors['divide_by']
    errors[:, 3][np.where(errors[:, 3] > angular_norm_factors['set_to_inf_before_dividing'])] = np.inf
    errors[:, 3][np.isinf(errors[:, 3])] = np.nan
    errors[:, 3] /= angular_norm_factors['divide_by']
    errors = scipy.stats.gmean(errors, axis=1)
    return errors

In [18]:
# Create a motif from the CN 16 clusters.
# There were not many of these so motif extraction did not identify a motif with CN 16.
# We make the assumption that all the clusters with CN 16 have the same prototypical structure.

cluster_indices_CN16 = np.where(cluster_cns == 16)[0]
print(len(cluster_indices_CN16))

errors_CN16 = []
for i in cluster_indices_CN16:
    try:
        errors_CN16.append(
            np.load(f"../motifextraction/examples/motifextraction/metallic_glass/data/errors/{i}_errors.npy")
        )
    except FileNotFoundError:
        errors_CN16.append(None)
print(len(errors_CN16))

110
110


In [26]:
# Find the cluster that is most similar to all the other clusters with CN 16
best = (np.inf, None)
for i, errors in enumerate(errors_CN16):
    if errors is None:
        continue
    errors = errors[cluster_indices_CN16]
    errors = calculate_gmean(errors)
    m = np.nanmean(errors)
    if m < best[0]:
        best = (m, cluster_indices_CN16[i])
print(best[1])

2214


/home/group/anaconda2/envs/py3/lib/python3.6/site-packages/scipy/stats/stats.py:314: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(a)


In [31]:
cluster = Cluster(filename=cluster_path / "2214.xyz")
atom_positions = []
for c in cluster_indices_CN16:
    c = Cluster(filename=cluster_path / f"{c}.xyz")
    data = AlignedData.from_mapping(align(c.filename, cluster.filename))
    if data.successful:
        atom_positions.append(np.array(data.aligned_model.positions))
atom_positions = np.array(atom_positions)

In [32]:
print(atom_positions.shape)
atom_positions = np.mean(atom_positions, axis=0)
print(atom_positions.shape)

(103, 17, 3)
(17, 3)


In [46]:
with open("CN_16_averaged_manually.xyz", 'w') as f:
    f.write(f"{len(atom_positions)}\n")
    f.write(f"comment\n")
    for p in atom_positions:
        f.write(f"Si {p[0]} {p[1]} {p[2]}\n")

In [47]:
c = Cluster(filename="./CN_16_averaged_manually.xyz")
print(c.CN)
print(c.vp_index)

16
(0, 1, 10, 5, 0, 0, 0, 0, 0, 0)
